In [ ]:
################################
# EvoMan FrameWork - V1.0 2016 #
# Author: Karine Miras         #
# karine.smiras@gmail.com      #
################################

# imports framework
import sys, os
from numpy.random import randint
import numpy as np
sys.path.insert(0, 'evoman') 
from environment import Environment
from demo_controller import player_controller
import random

experiment_name = 'dummy_demo'
if not os.path.exists(experiment_name):
    os.makedirs(experiment_name)

# global training parameters
n_hidden_neurons = 3
n_population = 10
n_generations = 30
min_value = -1
max_value = 1

# choose this for not using visuals and thus making experiments faster
headless = True
if headless:
    os.environ["SDL_VIDEODRIVER"] = "dummy"


# initializes environment with ai player using random controller, playing against static enemy
env = Environment(experiment_name=experiment_name,
                  enemies=[2],
                  playermode="ai",
                  player_controller=player_controller(n_hidden_neurons),
                  enemymode="static",
                  level=1,
                  speed="fastest")

def initialise_population(n_pop, weights):
    """ Function to randomly create the original population, 
        is called in the evolution function.
        returns list of length size of populations with weights (83) inside them
    """
    init_pop = np.random.uniform(min_value, max_value, (n_pop, weights))

    return init_pop


def evalualate_pop(population: np.ndarray):
    '''
    Tests given population on enemy, 
    Returns fitness values i na list of length size of populations
    '''
    res_pop = []
    for pop in population:
        fitness, player_life, enemy_life, time = env.play(pcont=pop)
        res_pop.append(fitness)
    return res_pop

def get_probability(fitness_of_population):
    """
    Get the probabilities for the roulette wheel selection.
    NOT USED AT THE MOMENT
    """
    fitness = fitness_of_population  #evalualate_pop(population)- repeats the process
    total_fit = float(sum(fitness))
    relative_fitness = [c/total_fit for c in fitness]
    probabilities = [sum(relative_fitness[:i+1]) for i in range((len(relative_fitness)))] ### Returns the probabilities relevant to individual fitness, Imagine this as the actual roulette wheel
    return probabilities

def roulette_wheel(population, probabilities, number_to_mate):
    """
    Roulette wheel selection
    NOT USED AT THE MOMENT
    """
    choice = []
    for n in range(number_to_mate):
        r = random.random() ## random number between 0.0 and 1.0
        for i, individual in enumerate(population): 
            if r <= probabilities[i]:
                choice.append(list(individual))
                break
    return choice
    
def tournament_parent_selection(population, fitness_of_population, size=2):
    """
    Function to select best individuals from the population in tournament style:
    Needs the current population, fitness, and how many will participate
    returns one selected parent so needs to be used in a loop (as is used in the evolution function)
    """
    fitness_of_tourney = fitness_of_population
    rand_selection = randint(len(population))
    for i in randint(0, len(population), size-1):
        if fitness_of_population[i] > fitness_of_population[rand_selection]:
            rand_selection = i
    return population[rand_selection]

def two_bit_cross_over(parent_1, parent_2, cross_site = list):
    """
    This function applies crossover for the case of two parents.
    Requires: parents_1 and parent_2 - both parents in list form (like from roulette) OR each parent seperately like in case of tournament selection
    Requires: cross_site = list of 2 elements, both elements indicating where the crossover will take place
    Returns: 2 children 
    """
    ### 2- bit crossover, from paper: https://www.researchgate.net/publication/220485962_A_Novel_Crossover_Operator_for_Genetic_Algorithms_Ring_Crossover#pf2
    ### Advantage: an advantage of having more crossover points is that the problem space may be searched more thoroughly. In two-point crossover (TPC), 
        # two crossover points are chosen and the contents between these points are exchanged between two mated parents [5, 6] In figure 2, 
        # the arrows indicate the crossover points.
    columns = len(parent_1) # Get length of parent

    child_1 = np.zeros(columns) ### Set children to 0 in length of parent
    child_2 = np.zeros(columns)

    for i in range(columns):
        if i >= cross_site[0] and i <= cross_site[1]:
            child_1[i] = parent_2[i]
            child_2[i] = parent_1[i]
        else:
            child_1[i] = parent_1[i]
            child_2[i] = parent_2[i]

    ### Example: for 2 arrays of parent_1 = np.array([1, 1, 1, 1, 1, 1, 1, 1])
    ###                          parent_2 = np.array([0, 0, 0, 0, 0, 0, 0, 0])
    ###                 returns  child 1 : [1. 1. 1. 0. 0. 0. 0. 1.]    child 2: [0. 0. 0. 1. 1. 1. 1. 0.]
    ###                 Where middle 3 elements get swaped

    cross_res = []
    cross_res.append(child_1)
    cross_res.append(child_2)
    cross_res = np.array(cross_res)

    return cross_res

def mutation(cross_over_result):
    """
    Mutation to change gene in each offspring randomly
    Requires: results from the cross over

    It loops through each offspring and adds random uniform number which si then added to the gene with random index.
    """
    for idx in range(0, 1): ### changed from = for idx in range(cross_over_result.shape[0]):
        # # The random value to be added to the gene
        random_value = np.random.uniform(-1.0, 1.0, 1)
        # #random_index = np.random.choice()
        # cross_over_result[np.random.choice(cross_over_result)] = cross_over_result[np.random.choice(cross_over_result)] + random_value
        get_rid_of_random_ele = random.choices(cross_over_result, k=len(cross_over_result)-1)
        get_rid_of_random_ele.insert(random.randrange(0, len(get_rid_of_random_ele)-1), random_value)

    return get_rid_of_random_ele

def get_worst_fitness(all_fitnesses):
    """ MOST LIKELY USELESS AND NOT NEEDED
    """
    small_1, small_2, *_ = np.partition(all_fitnesses,1)
    two_worst = []
    two_worst.append(small_1)
    two_worst.append(small_2)
    return two_worst



# Start evolution algorithm
# _____________________________________________________________________________________________________________________________________________
def evo_run(n_generations: int, n_population: int, n_hidden_neurons: int):
    stats = []
    max_fitness_for_gen = []
    average_fitness_for_gen = []

    n_weights = (env.get_num_sensors()+1)*n_hidden_neurons+(n_hidden_neurons+1)*5 # weights and biases needed for NN
    #print(f"Weights and biases for the NN: {n_weights}")
    # for first generation initialize a random population
    init_pop = initialise_population(n_population, n_weights)
    for g in range(0, n_generations):
        print(f"Current generation: {g}") 
        
        next_pop = list()
        all_fit = evalualate_pop(init_pop)
        #print(f"ALL FITNESS: {all_fit}")
        
        best_index = np.argmax(all_fit) ### Index of best fitness out of population
        #print(f"BEST INDEX: {best_index}")
        best_fit = init_pop[best_index] ### Individual with best fitness
        # print(f"BEST FITNESS: {best_fit}")
        # print('\n')

        stdev = np.std(all_fit)
        
        average = np.mean(all_fit)
        print(f"AVERAGE fitness: {average}")

        for i in range(n_population // 2): # For 10 individuals there will be: 
            p1 = tournament_parent_selection(init_pop, all_fit) # 5 p1
            p2 = tournament_parent_selection(init_pop, all_fit) # 5 p2
            # print(f"PARENT 1: {p1}")
            # print(f"PARENT 2: {p2}")

            ### crossover
            c1, c2 = two_bit_cross_over(p1, p2, cross_site = [60, 69]) # For population of 10 there would be 5 child 1 and 5 child 2
            # print(f"CHILD 1: {c1}")
            # print(f"CHILD 2: {c2}")
            
            # Next generation
            next_pop.extend([c1, c2]) # Adds child 1 and child 2 at the end of next_pop list
            #print(f"AFTER EXTEND: {next_pop}")

        ### Mutation
        for individual in next_pop:
            mutation(individual)
            
        # Replace Population
        init_pop = next_pop
        #save data
        # when loop is finished return gene with best fitness

        stats.append([np.max(all_fit), np.min(all_fit), np.mean(all_fit)])
        max_fitness_for_gen.append(best_fit)
        average_fitness_for_gen.append(average)

        # Print the results of 
    for gen in range(len(stats)):
        print("\n")
        print(f"Generation {gen} maximum fitness: {stats[gen][0]}, minimum fitness: {stats[gen][1]}, AVERAGE FITNESS: {stats[gen][2]}")


# Notes on Parent Selection: 
- care should be taken to prevent one extremely fit solution from taking over the entire population in a few generations.
- Maintaining good diversity in the population is extremely crucial for the success of a GA.
- This taking up of the entire population by one extremely fit solution is known as premature convergence.

## Roulette wheel Selection
- Process of selection is repeated for each parent. 
- fitter individual has a greater pie on the wheel and therefore a greater chance
    ## Process: 
    - Get sum of fitness
    - generate a random number between 0 and the sum
    - Starting from the top of the population, keep adding the finesses to the partial sum P, till P<Sum.
    - The individual for which P exceeds S is the chosen individual.

## Tournament Selection
- Only the fittest candidate amongst those selected candidates is chosen and is passed on to the next generation
- Selection preassure can be added (probabilistic measure of a candidate’s likelihood of participation in a tournament)
    ## Process
    - Select k individuals from the population and perform a tournament amongst them
    - Select the best individual from the k individuals
    - Repeat process 1 and 2 until you have the desired amount of population

# Mutation
- Crossover enables the algorithm to extract the best genes from different individuals and recombine them into potentially superior children. 
- Mutation adds to the diversity of a population and thereby increases the likelihood that the algorithm will generate individuals with better fitness values.
- Mutation is a genetic operator used to maintain genetic diversity from one generation of a population of genetic algorithm chromosomes to the next. It is analogous to biological mutation.
    ## Applied :
    - applied to the results of the crossover for diversity.



In [ ]:
evo_run(3, 100, 3)